## What's happening in SA

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get("https://www.sanantonio.gov/Health/News/Alerts/CoronaVirus#281783804-public-info-")

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find_all("table",{"class":"covidData"})

In [82]:
def make_df(table_body):
    data = []
    rows = table_body.find_all('tr')

    for row in rows:
        head = row.find('th').text
        head = head.strip()

        col = row.find('td').text
        data.append([head, col])
    return pd.DataFrame(data)

### Bexar County

In [112]:
table_body = table[0].find('tbody')
make_df(table_body)

,0,1
0,Total Confirmed cases Bexar County,157
1,Travel-related,61
2,Close contact,21
3,Community transmission,55
4,Under investigation,20


### Hopitalizations

In [147]:
table_body = table[1].find('tbody')
rows = table_body.find_all('tr')

for row in rows:
    head = row.find('th')
#     print('this is the head:', head)
    if head != None:
        head = head.text.strip()
        print('head:', head)
#         print(head)
    col = row.find('td').text.strip()
    print('col :', col)
    print('')
#     data.append([head, col])
# data

head: Ever Hospitalized
col : 50

col : Current Hospitalizations*

head: In Intensive Care Unit
col : In Intensive Care Unit
12


On Ventilator
11



            *Source: STRAC

head: In Intensive Care Unit
col : 12

head: On Ventilator
col : 11



### Confirmed Cases by Gender

In [148]:
table_body = table[2].find('tbody')
make_df(table_body)

,0,1
0,Male,85
1,Female,72


### Confirmed Cases by Age

In [144]:
table_body = table[3].find('tbody')
make_df(table_body)

,0,1
0,0-19,7
1,20-29,26
2,30-39,19
3,40-49,40
4,50-59,24
5,60-69,23
6,70-79,15
7,80-89,3


### Death Case Information

In [145]:
table_body = table[4].find('tbody')
make_df(table_body)

,0,1
0,Total Deaths,5


### Death Cases by Gender

In [146]:
table_body = table[5].find('tbody')
make_df(table_body)

,0,1
0,Male,0
1,Female,5


### Death Cases by Age Group

In [150]:
table_body = table[6].find('tbody')
make_df(table_body)

,0,1
0,0-19,0
1,20-29,0
2,30-39,0
3,40-49,2
4,50-59,1
5,60-69,1
6,70-79,0
7,80-89,1


### Testing

In [151]:
table_body = table[7].find('tbody')
make_df(table_body)

,0,1
0,Negative,538
1,Positive,57
2,Inconclusive,1
3,Total tested by Metro Health Lab,596


In [153]:
table_body = table[8].find('tbody')
make_df(table_body)

AttributeError: 'NoneType' object has no attribute 'text'